In [96]:
import pandas as pd
import numpy as np

# Neural Network Class

In [97]:
class NeuralNetwork:
    
    def __init__(self,inputNodes,hiddenList,hiddenActivations,outputNodes=1,outputActivation="SIGMOID"):
        self.inputNodes=inputNodes
        self.outputNodes=outputNodes
        self.hiddenList=[inputNodes]+hiddenList+[outputNodes]
        # Nodes in each Layer (Also stores corresponding to input and output layer)
        self.Activations=["None"]+hiddenActivations+[outputActivation]
        # Activation Functions (Also stores corresponding to input and output layer)
        self.weight=[]
        self.os=[]
        self.net=[]
        self.deltas=[]
        self.loss=0
        self.trainsteps=0
        self.epoch=0
    
    # Initialise weight Matrices Randomly
    def initialiseWeight(self):
        layers=len(self.hiddenList)
        self.trainsteps=0
        self.epoch=0
        
        for i in range(1,layers,1):
            self.weight.append(np.random.randn(self.hiddenList[i],self.hiddenList[i-1]+1))
    
    # Applies Activation Function
    def applyActivation(self,vector,activation="None"):
        if(activation=="RELU"):
            return vector*(vector>=0)
        elif(activation=="SIGMOID"):
            return (1.0/(1+np.exp(-vector)))
        else:
            return 1*vector
    
    def append1(self,x):
        [instances,attr]=x.shape
        bias=np.ones([instances,1])
        return np.concatenate((bias,x),axis=1)
    
    # Forward Pass a batch through the network
    def forwardPass(self,batchX):
        
        layers=len(self.weight)
        self.os=[self.append1(batchX)]
        self.net=[self.append1(batchX)]
                
        for i in range(layers):
            x=self.os[i]
            netj=np.matmul(x,self.weight[i].T)
            o=(self.applyActivation(netj,self.Activations[i+1]))
            if(i==layers-1):
                self.net.append(netj)
                self.os.append(o)
            else:
                self.net.append(self.append1(netj))
                self.os.append(self.append1(o))
    
    # Euclidean Loss -> if Changed, gradient J wrt o must be updated
    def lossfunction(self,pred,y):
        return (pred-y)**2
    
    # Function returning loss on batch
    def getLoss(self,batchX,batchY):
        self.forwardPass(batchX)
        layers=len(self.os)
        finalOutput=self.os[layers-1]
        return np.sum(self.lossfunction(finalOutput,batchY))

    # Gradient of o wrt net 
    def gradient_o_net(self,o,net,activation="None"):
        if(activation=="RELU"):
            return (net>=0)
        elif(activation=="SIGMOID"):
            return o*(1-o)
        else:
            return (o==o)
    
    def gradient_o_net_layer(self,layerNo):
        return self.gradient_o_net(self.os[layerNo],self.net[layerNo],self.Activations[layerNo])
    
    # Gradient of J wrt o
    def gradient_J_o(self,o,y):
        return 2*(o-y)        

    # Backward Pass a batch through the network updating Weight Matrices   
    def backwardPass(self,batchX,batchY,learningRate=0.001):
                
        layers=len(self.os)
#         m=float(1)
        m=float((batchX.shape)[0])
        self.deltas=[]
        
        finalDelta=self.gradient_J_o(self.os[layers-1],batchY)*self.gradient_o_net_layer(layers-1)        
        self.deltas.append(finalDelta)
        weightMatrixLen=len(self.weight)
        
        for i in range(layers-2,0,-1):
            weightMatrixLen-=1
            current_o_net=self.gradient_o_net_layer(i)
            current_delta=np.matmul(self.deltas[0],self.weight[weightMatrixLen])
            current_delta=current_delta*current_o_net
            [_,attributes]=current_delta.shape
            self.deltas=[current_delta[:,1:attributes]]+self.deltas
        
        layers=len(self.weight)
         
        for i in range(layers-1,-1,-1):
            del_w=np.matmul(self.deltas[i].T,self.os[i])
            self.weight[i]-=(learningRate/m)*del_w

    # training the NN
    def train(self,X,Y,learningRate=0.001,batchMode=False,batchSize=100,epsilon=0.00001,epochsToRun=500):
        
        [instances,attributes]=X.shape
        [_,outputs]=Y.shape
        
        if(batchMode==False):
            batchSize=instances
        
        Trained=False
        prevLoss=self.getLoss(X,Y)
        
        while(not Trained and self.epoch<epochsToRun):
            self.epoch+=1
            
            cur=0
            while(cur<instances):
                uplim=min(cur+batchSize,instances)
                batchX=X[cur:uplim,0:attributes]
                batchY=Y[cur:uplim,0:outputs]
                cur=uplim
                self.forwardPass(batchX)
                self.backwardPass(batchX,batchY,learningRate)
                self.trainsteps+=1
            
            curLoss=self.getLoss(X,Y)
            
            if(abs(curLoss-prevLoss)<epsilon):
                Trained=True
            
            prevLoss=curLoss
            if(self.epoch%20==0):
                print (self.epoch,prevLoss)
                print (self.getAccuracy(X,Y))
        
    def predict(self,X):
        self.forwardPass(X)
        layers=len(self.os)
        return self.os[layers-1]
    
    def getAccuracy(self,X,Y):
        pred=self.predict(X)
        instances=float(X.shape[0])
        
        pred=(pred>=0.5).astype(int)
        
        return (np.sum(pred==Y))/instances
        
            
            
        

# Loading Data

In [98]:
# xtrain=pd.read_csv("Dataset/NN/toy_data/toy_trainX.csv",header=None,sep=',').values
# ytrain=pd.read_csv("Dataset/NN/toy_data/toy_trainY.csv",header=None,sep=',').values
# xtest=pd.read_csv("Dataset/NN/toy_data/toy_testX.csv",header=None,sep=',').values
# ytest=pd.read_csv("Dataset/NN/toy_data/toy_testY.csv",header=None,sep=',').values

In [99]:
xtrain=pd.read_csv("Dataset/NN/mnist_data/MNIST_train.csv",header=None,sep=',').values
# ytrain=pd.read_csv("Dataset/NN/toy_data/toy_trainY.csv",header=None,sep=',').values
ytrain=xtrain[:,784:785]
xtrain=xtrain[:,0:784]
xtest=pd.read_csv("Dataset/NN/mnist_data/MNIST_test.csv",header=None,sep=',').values
ytest=xtest[:,784:785]
xtest=xtest[:,0:784]
ytrain=(ytrain==6).astype(int)
ytest=(ytest==6).astype(int)
xtrain=xtrain/255.0
xtest=xtest/255.0
# print(np.sum(xtrain))
# print(np.sum(xtest))
# ytest=pd.read_csv("Dataset/NN/toy_data/toy_testY.csv",header=None,sep=',').values

In [100]:
# print(xtest)

In [101]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(10000, 784)
(10000, 1)
(3600, 784)
(3600, 1)


# Testing NN

In [102]:
# NN_obj=NeuralNetwork(xtrain.shape[1],[5,5],["SIGMOID","SIGMOID"],ytrain.shape[1])
# NN_obj=NeuralNetwork(xtrain.shape[1],[5,5],["RELU","RELU"],ytrain.shape[1])
# NN_obj=NeuralNetwork(xtrain.shape[1],[5],["SIGMOID"],ytrain.shape[1])
NN_obj=NeuralNetwork(xtrain.shape[1],[100],["RELU"],ytrain.shape[1])

In [103]:
NN_obj.initialiseWeight()

In [104]:
for weights in NN_obj.weight:
    print(weights.shape)

(100, 785)
(1, 101)


In [105]:
NN_obj.getLoss(xtrain,ytrain)

5536.692429033992

In [106]:
NN_obj.train(X=xtrain,Y=ytrain,learningRate=0.1,batchMode=True,batchSize=100,epochsToRun=200)

20 472.9736742936408
0.9508
40 308.371705092701
0.9678
60 230.90141483529518
0.976
80 196.7832091166643
0.9799
100 181.2641463879188
0.9818
120 172.0896827378161
0.9826


KeyboardInterrupt: 

In [95]:
print (NN_obj.getAccuracy(xtrain,ytrain))
print (NN_obj.getAccuracy(xtest,ytest))

0.9898
0.9844444444444445
